# master simulations

All simulations needed for the Scientific Report paper / supp material:
1. taking a bunch of images from Serre & al containing or not an animal, plus the laboratory set,
2. compute edges and
3. then doing statistics on that
4. to see if we can classify them.


## simulations for the manuscript

````
    rm -fr mat/edges/ms_* mat/ms_* figures/edges/ms_* figures/ms_* figures/figure*
    frioul_batch  -n "14,15,16"  -M 36 'python experiment_ms.py'
````

In [1]:
%load_ext autoreload
%autoreload 2
%cd -q ../test/

In [2]:
%%writefile experiment_ms.py
#! /usr/bin/env python
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""

paper:
1. taking a bunch of images from Serre & al containing or not an animal, plus the laboratory set,
2. compute edges and
3. then doing statistics on that
4. to see if we can classify them.

rm -fr mat/edges/srep_* mat/srep_* figures/edges/srep_* figures/srep_* figures/edges/srep_*
frioul_batch  -n "14,15,16"  -M 36 'python experiment_ms.py'
frioul_batch  -n "17,18,19"  -M 36 'python experiment_ms.py'


"""
__author__ = "(c) Laurent Perrinet INT - CNRS"

import matplotlib
matplotlib.use("Agg") # agg-backend, so we can create figures without x-server (no PDF, just PNG etc.)

import os
import numpy as np
import pickle
exp = 'srep'
verb = False

## TODO precision (accross images) as alpha value?
from SparseEdges import EdgeFactory
ef = EdgeFactory('default_param.py')
ef.pe.verbose = 50

features = ['first', 'first_rot', 'first_full', 'first_chevron']
features = ['first', 'first_rot', 'full', 'chevron']

#######################################################################
#                 table 1 : classification results                    #
#######################################################################
list_db = [['serre07_distractors', 'serre07_targets'],
           ['serre07_distractors', 'treeshrew'],
           ['serre07_distractors', 'laboratory'],
          ]
if verb: print(15*'-~-',  ' DOING THE NUMBER CRUNCHING ', 15*'-~-')
for feature in features:
    if verb: print(5*'-~-',  'feature: ', feature, 5*'-~-')
    for databases in list_db:
        if verb: print(5*'-*-',  'databases: ', databases, 5*'-*-')
        # without noise
        ef.svm(exp, databases=databases, feature=feature)
        # with noise
        ef.svm(exp + '_noise', databases=databases, feature=feature, noise=ef.pe.noise)

m, s =  np.zeros((4, 2, 2)), np.zeros((4, 2, 2))
for i, feature in enumerate(features):
    for j, databases in enumerate(list_db):
        try:
            category = '_SVM_' + databases[0] + '_' + databases[1]
            m[i, j, 0] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').mean()*100
            s[i, j, 0] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').std()*100
            category = '_noise_SVM_' + databases[0] + '_' + databases[1]
            m[i, j, 1] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').mean()*100
            s[i, j, 1] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').std()*100
        except Exception as e:
            print(e)
print(60*'-')
print('table 1: classification results')
print(60*'-')
print('\% Categories & FO & FO_rot & SO & CM ')
print(60*'-')
print('Non-animal vs Artificial             & ' + '%.0f' % m[0, 1, 0] + '\pm %.0f' % s[0, 1, 0] + '\% & ' + '%.0f' % m[1, 1, 0] + '\pm %.0f' % s[1, 1, 0]  + '\% & ' + '%.0f' % m[2, 1, 0] + '\pm %.0f' % s[2, 1, 0]  + '\% & ' + '%.0f' % m[3, 1, 0] + '\pm %.0f' % s[3, 1, 0]  + '\% \\\\')
print('Non-animal vs Animal                 & ' + '%.0f' % m[0, 0, 0] + '\pm %.0f' % s[0, 0, 0] + '\% & ' + '%.0f' % m[1, 0, 0] + '\pm %.0f' % s[1, 0, 0]  + '\% & ' + '%.0f' % m[2, 0, 0] + '\pm %.0f' % s[2, 0, 0] + '\% & ' + '%.0f' % m[3, 0, 0] + '\pm %.0f' % s[3, 0, 0]   + '\% \\\\')
print('Non-animal (noise) vs Animal (noise) & ' + '%.0f' % m[0, 0, 1] + '\pm %.0f' % s[0, 0, 1] + '\% & ' + '%.0f' % m[1, 0, 1] + '\pm %.0f' % s[1, 0, 1]  + '\% & ' + '%.0f' % m[2, 0, 1] + '\pm %.0f' % s[2, 0, 1] + '\% & ' + '%.0f' % m[3, 0, 1] + '\pm %.0f' % s[3, 0, 1]   + '\% \\\\')


#######################################################################
#                 table 2 : classification results                    #
#######################################################################
r = np.zeros((4,4))
categories = ['_close', '_far', '_head', '_whole']
for i, feature in enumerate(features):
#, 'first_chevron', 'first_full']):
    if verb: print(5*'-~-',  'feature: ', feature, 5*'-~-')
    for j, category in enumerate(categories):
        try:
            if verb: print(5*'-*-',  'category: ', category, 5*'-*-')
            ef.svm(exp, databases=['serre07_distractors' + category, 'serre07_targets' + category], feature=feature)
            r[i, j] = np.load('cache_dir/' + exp + '_SVM_serre07_distractors' + category + '_serre07_targets' + category + '_' + feature + '.npy').mean()*100
        except Exception as e:
            print(e)
print(60*'-')
print('table 2: sub-categories')
print(60*'-')
print('\% Category & Human & Serre07 & FO & FO_rot & SO & CM ')
print(60*'-')
print('Head  & 78\% & 92\% & ' + '%.1f' % r[0, 0] + '\% & ' + '%.1f' % r[1, 0] + '\% & ' + '%.1f' % r[2, 0] + '\% & ' + '%.1f' % r[3, 0] + '\% \\\\')
print('Close & 82\% & 90\% & ' + '%.1f' % r[0, 1] + '\% & ' + '%.1f' % r[1, 1] + '\% & ' + '%.1f' % r[2, 1] + '\% & ' + '%.1f' % r[3, 1] + '\% \\\\')
print('Whole & 71\% & 79\% & ' + '%.1f' % r[0, 2] + '\% & ' + '%.1f' % r[1, 2] + '\% & ' + '%.1f' % r[2, 2] + '\% & ' + '%.1f' % r[3, 2] + '\% \\\\')
print('Far   & 52\% & 68\% & ' + '%.1f' % r[0, 3] + '\% & ' + '%.1f' % r[1, 3] + '\% & ' + '%.1f' % r[2, 3] + '\% & ' + '%.1f' % r[3, 3] + '\% \\\\')


#######################################################################
#                 table 3 : more databases                            #
#######################################################################
ef = EdgeFactory('default_param.py')
ef.pe.verbose = 50
ef.N_X = 244
ef.N_Y = 244
ef.init()

list_db = [['holle_distractors', 'holle_targets'],
           ['holle_distractors', 'laboratory'],
          ]
for feature in features:
    if verb: print(5*'-~-',  'feature: ', feature, 5*'-~-')
    for databases in list_db:
        if verb: print(5*'-*-',  'databases: ', databases, 5*'-*-')
        # without noise
        ef.svm(exp, databases=databases, feature=feature)
        # with noise
        ef.svm(exp + '_noise', databases=databases, feature=feature, noise=ef.pe.noise)

m, s =  np.zeros((4, 2, 2)), np.zeros((4, 2, 2))
for i, feature in enumerate(features):
    #for j, category in enumerate(['_SVM_serre07_distractors_serre07_targets', '_noise_SVM_serre07_distractors_serre07_targets', '_SVM_serre07_distractors_laboratory']):
    for j, databases in enumerate(list_db):
        try:
            category = '_SVM_' + databases[0] + '_' + databases[1]
            m[i, j, 0] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').mean()*100
            s[i, j, 0] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').std()*100
            category = '_noise_SVM_' + databases[0] + '_' + databases[1]
            m[i, j, 1] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').mean()*100
            s[i, j, 1] = np.load('cache_dir/' + exp + category + '_' + feature + '.npy').std()*100
        except Exception as e:
            import sys, traceback
            tb = sys.exc_info()[2]#, sys.exc_traceback.print_stack()
            print(e, traceback.print_tb(tb))
print(60*'-')
print('table 3: more classification results')
print(60*'-')
print('\% Categories & FO & FO_rot & SO & CM ')
print(60*'-')
print('Non-animal vs Artificial             & ' + '%.0f' % m[0, 1, 0] + '\pm %.0f' % s[0, 1, 0] + '\% & ' + '%.0f' % m[1, 1, 0] + '\pm %.0f' % s[1, 1, 0]  + '\% & ' + '%.0f' % m[2, 1, 0] + '\pm %.0f' % s[2, 1, 0]  + '\% & ' + '%.0f' % m[3, 1, 0] + '\pm %.0f' % s[3, 1, 0]  + '\% \\\\')
print('Non-animal vs Animal                 & ' + '%.0f' % m[0, 0, 0] + '\pm %.0f' % s[0, 0, 0] + '\% & ' + '%.0f' % m[1, 0, 0] + '\pm %.0f' % s[1, 0, 0]  + '\% & ' + '%.0f' % m[2, 0, 0] + '\pm %.0f' % s[2, 0, 0] + '\% & ' + '%.0f' % m[3, 0, 0] + '\pm %.0f' % s[3, 0, 0]   + '\% \\\\')
print('Non-animal (noise) vs Animal (noise) & ' + '%.0f' % m[0, 0, 1] + '\pm %.0f' % s[0, 0, 1] + '\% & ' + '%.0f' % m[1, 0, 1] + '\pm %.0f' % s[1, 0, 1]  + '\% & ' + '%.0f' % m[2, 0, 1] + '\pm %.0f' % s[2, 0, 1] + '\% & ' + '%.0f' % m[3, 0, 1] + '\pm %.0f' % s[3, 0, 1]   + '\% \\\\')

## TODO precision (accross images) as alpha value?
print('~~~~ Exiting gracefully ~~~~')


Overwriting experiment_ms.py


In [3]:
%run experiment_ms.py

index 2 is out of bounds for axis 1 with size 2
index 2 is out of bounds for axis 1 with size 2
index 2 is out of bounds for axis 1 with size 2
index 2 is out of bounds for axis 1 with size 2
------------------------------------------------------------
table 1: classification results
------------------------------------------------------------
\% Categories & FO & FO_rot & SO & CM 
------------------------------------------------------------
Non-animal vs Artificial             & 88\pm 2\% & 72\pm 4\% & 98\pm 1\% & 88\pm 2\% \\
Non-animal vs Animal                 & 76\pm 2\% & 70\pm 3\% & 84\pm 2\% & 77\pm 2\% \\
Non-animal (noise) vs Animal (noise) & 73\pm 3\% & 70\pm 3\% & 79\pm 3\% & 70\pm 3\% \\
------------------------------------------------------------
table 2: sub-categories
------------------------------------------------------------
\% Category & Human & Serre07 & FO & FO_rot & SO & CM 
------------------------------------------------------------
Head  & 78\% & 92\% & 77.5\%


## more simulations to test the effect of parameters


In [4]:
%%writefile experiment_animals.py
#/usr/bin/env python
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""

Doing an optional stuff: taking a bunch of images from Kirchner & Thorpe or
Serre et al., 2007, compute edges and then doing statistics on that to see if
we can classify them.

rm -fr mat/edges/classifier_*npy mat/classifier_*npy figures/edges/classifier_* figures/classifier_*
! frioul_batch  -n "14,15,16"  -M 36 'python experiment_animals.py'
! frioul_batch  -n "11,12,13,14,15,16"  -M 72 'python experiment_animals.py'

"""
__author__ = "(c) Laurent Perrinet INT - CNRS"

import matplotlib
matplotlib.use("Agg") # agg-backend, so we can create figures without x-server (no PDF, just PNG etc.)

import os
import numpy as np
import pickle

exp, paramfile = 'supp', 'default_param.py'

## TODO precision (accross images) as alpha value?

from SparseEdges import EdgeFactory

features = ['first', 'first_rot', 'full', 'chevron']

#! parameter scan
#!---------------
# generic experiments
for feature in features:
    ef = EdgeFactory(paramfile)
    for ef.pe.N_r in np.arange(1, 10):
        ef.svm(exp, feature=feature, opt_notSVM='_N_r_' + str(ef.pe.N_r))
        
    ef = EdgeFactory(paramfile)
    for svm_KL_m in np.logspace(-.75, .75, 10, base=10)*ef.pe.svm_KL_m:
        ef.pe.svm_KL_m = svm_KL_m
        ef.svm(exp, feature=feature, opt_SVM='_precomputed_KL_m_' + str(ef.pe.svm_KL_m).replace('.', '_'), kernel='precomputed')
    
    ef = EdgeFactory(paramfile)
    for KL_type in ['sKL', 'JSD', 'KL']:
        ef.svm(exp, feature=feature, opt_SVM='_precomputed_KL_type_' + KL_type, KL_type=KL_type, kernel='precomputed')

    for kernel in ['rbf', 'precomputed']:
        for svm_test_size in np.linspace(.1, 1., 10, endpoint=False):
            ef = EdgeFactory(paramfile)
            ef.pe.svm_test_size = svm_test_size
            ef.svm(exp, feature=feature, opt_SVM='_testing_' + kernel + '_test_size_' + str(ef.pe.svm_test_size).replace('.', '_'), kernel=kernel)

    for svm_log in [True, False]:
        ef = EdgeFactory(paramfile)
        ef.pe.svm_log = svm_log
        ef.svm(exp, feature=feature, opt_notSVM='_svm_log_' + str(ef.pe.svm_log))

    for svm_norm in [True, False]:
        for feature in ['first', 'first_rot', 'full', 'chevron']:
            for databases in (['serre07_distractors', 'serre07_targets'], ['serre07_distractors', 'laboratory']): # ['serre07_targets', 'serre07_distractors']
                ef = EdgeFactory(paramfile)
                ef.pe.svm_norm = svm_norm
                ef.svm(exp, feature=feature, databases=databases)
                ef.svm(exp + '_noise', feature=feature, databases=databases, noise=ef.pe.noise)

    for svm_tol in np.logspace(-9., 0., 9, base=10, endpoint=False):
        ef = EdgeFactory(paramfile)
        ef.pe.svm_tol = svm_tol
        ef.svm(exp, feature=feature, opt_SVM='_svm_tol_' + str(ef.pe.svm_tol).replace('.', '_'))

# experiments specific to the way we compute the histograms
for feature in ['full', 'chevron', 'first_chevron', 'first_full']:
#     for ef.pe.N_Dtheta in [1, 24, 48]:
#         ef.svm(exp, feature=feature, opt_notSVM='_N_Dtheta_' + str(ef.pe.N_Dtheta))
#     ef.pe.N_Dtheta = pe.N_Dtheta

    for d_min in np.logspace(-1., 0., 9, base=10, endpoint=False)*ef.pe.d_max:
        ef = EdgeFactory(paramfile)
        ef.pe.d_min = d_min
        ef.svm(exp, feature=feature, opt_notSVM='_d_min_' + str(ef.pe.d_min).replace('.', '_'))

    for d_max in np.linspace(ef.pe.d_min*1.1, ef.pe.d_max*2, 10):
        ef = EdgeFactory(paramfile)
        ef.pe.d_max = d_max
        ef.svm(exp, feature=feature, opt_notSVM='_d_max_' + str(ef.pe.d_max).replace('.', '_'))

    for do_rank in [True, False]:
        for scale_invariant in [True, False]:
            for multiscale in [True, False]:
                for weight_by_distance in [True, False]:
                    ef = EdgeFactory(paramfile)
                    ef.pe.do_rank = do_rank
                    ef.pe.scale_invariant = scale_invariant
                    ef.pe.multiscale = multiscale
                    ef.pe.weight_by_distance = weight_by_distance
                    ef.svm(exp,  feature=feature, opt_notSVM='_do_rank_' + str(ef.pe.do_rank) + '_multiscale_' + str(ef.pe.multiscale) + '_scale_invariant_' + str(ef.pe.do_rank) + '_weight_by_distance_' + str(ef.pe.weight_by_distance))

#     for ef.pe.multiscale in [True, False]:
#         ef.svm(exp, opt_SVM=opt_SVM, feature=feature, opt_notSVM='_multiscale_' + str(ef.pe.multiscale))
#     ef.pe.multiscale = pe.multiscale

#     for ef.pe.kappa_phase in np.logspace(-3, 0, 5)*np.pi:
#         ef.svm(exp, opt_SVM=opt_SVM, feature=feature, opt_notSVM='_kappa_phase_' + str(ef.pe.kappa_phase).replace('.', '_'))
#     ef.pe.kappa_phase = pe.kappa_phase
# for ef.pe.kappa_phase in np.logspace(-2., 0., 5, base=10)*np.pi:
#     ef.svm(exp, opt_SVM=opt_SVM, opt_notSVM='_kappa_phase_' + str(ef.pe.kappa_phase))
# ef.pe.kappa_phase = pe.kappa_phase

#! cross-validation
#!-----------------
# list_db = [
#             ['serre07_distractors', 'serre07_targets'],
#             ['Yelmo', 'holle_distractors'],
#             ['treeshrew', 'holle_distractors'],
#             ['holle_targets', 'treeshrew'],
#             ['Yelmo', 'holle_targets'],
#             ['Yelmo', 'treeshrew'],
#             ['natural', 'holle_distractors'],
#             ['natural', 'holle_targets'],
#             ['laboratory', 'holle_distractors'],
#             ['laboratory', 'holle_targets'],
#             ['holle_distractors', 'holle_targets'],
#             ['holle_distractors', 'serre07_targets'],
#             ['serre07_distractors', 'holle_targets'],
#             ['natural', 'laboratory'],
#             ]
# for databases in list_db:#
#     ef.compare(exp, databases=databases)
#     ef.svm(exp, databases=databases)
#
#for databases in ['Yelmo', 'treeshrew', 'holle_distractors', 'holle_targets', 'natural', 'laboratory']:
#    ef.compare(exp, databases=[databases, databases])
#    ef.svm(exp, databases=[databases, databases])

# ef.svm(exp, databases=['holle_distractors', 'holle_targets', 'serre07_distractors', 'serre07_targets', 'natural', 'laboratory'])
# ef.svm(exp, databases=['serre07_distractors', 'serre07_targets', 'natural', 'laboratory'])
# ef.compare(exp, databases=['holle_distractors', 'holle_targets'])
# ef.svm(exp, databases=['holle_distractors', 'holle_targets'])
# ef.svm(exp, databases=['holle_distractors', 'holle_targets'], log=False)



## TODO:  would be interesting to see how f1-score changes with number of edges used, i.e. whether it settles down to that particular pattern or just jumps around.


## TODO precision (accross images) as alpha value?

print('~~~~ Exiting gracefully ~~~~')


Overwriting experiment_animals.py



## more simulations to test the effect of filters' shape


In [5]:
%%writefile experiment_testing.py
#! /usr/bin/env python
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""

Doing even more stuff: taking a bunch of images, computing edges and then doing
statistics on that.

rm -fr mat/edges/testing_* mat/testing_* figures/edges/testing_* figures/testing_*
frioul_batch  -n "14,15,16"  -M 36 'python experiment_testing.py'

"""
__author__ = "(c) Laurent Perrinet INT - CNRS"

import numpy as np

exp, paramfile = 'testing', 'default_param.py'
from NeuroTools.parameters import ParameterSet
pe = ParameterSet(paramfile)

from SparseEdges import EdgeFactory

def init_pe(pe):
    ef = EdgeFactory(pe)
    ef.pe.verbose = 5
    ef.pe.N = 256
    ef.pe.N_image = 80
    ef.init()
    return ef


# TODO: here, we are more interested in the processing of the database, not the comparison - use the correct function
# TODO : annotate the efficiency of different LogGabor bases (RMSE?)
# TODO: make a circular mask to avoid border effects coming with whitening...

#! comparing databases
#!--------------------
ef = init_pe(pe) # EF.EdgeFactory(edge=mp, pe=parameters)
# ef.compare('testing_vanilla')
ef.svm(exp + '_vanilla')

# TODO : make an experiment showing that using scale does not bring much
##! comparing representation parameters
##!------------------------------------
#! shorter log-gabor filters
pe_ = ParameterSet('default_param.py')
pe_.B_theta=pe.B_theta*2
ef = init_pe(pe_)
ef.svm(exp + '_short')

#! longer log-gabor filters
pe_= ParameterSet('default_param.py')
pe_.B_theta=pe.B_theta/2
ef = init_pe(pe_)
ef.svm(exp + '_long')

## other candidate parameters from class MatchingPursuit:
##    n_levels = 5, n_theta = 16, B_sf_ratio = 3.
pe_= ParameterSet('default_param.py')
pe_.n_theta=pe.n_theta*2
ef = init_pe(pe_)
ef.svm(exp + '_moretheta')

# is whitening important?
pe_= ParameterSet('default_param.py')
pe_.do_whitening=False
ef = init_pe(pe_)
ef.svm(exp + '_nowhite')

#! softy MP with a lower alpha value
pe_ = ParameterSet('default_param.py')
pe_.N=pe.N*2
pe_.MP_alpha = .25
ef = init_pe(pe_)
ef.svm(exp + '_smooth')

#! hard MP with a full alpha value
pe_ = ParameterSet('default_param.py')#
pe_.N=pe.N/2
pe_.MP_alpha = 1.
ef = init_pe(pe_) # EF.EdgeFactory(edge=mp, pe=parameters)
# ef.compare('testing_hard')
ef.svm(exp + '_hard')

##! comparing edge methods
##!-----------------------
## parameters from class EdgeFactory:
pe_ = ParameterSet('default_param.py')#
pe_.N = pe.N*4
ef = init_pe(pe_)
ef.svm(exp + '_moreN')


Overwriting experiment_testing.py


## running all scripts locally or remotely

In [ ]:
experiment_folder = 'PerrinetBednar15'

cluster = False
cluster = True

if cluster:
    try:
        from INT_cluster import Frioul
        k = Frioul(experiment_folder, N_jobs = 64)
    except Exception:
        cluster = False
else:
    def run_cmd(cmd, doit=True):
        import subprocess
        print ('⚡︎ Running ⚡︎ ', cmd)
        if doit:
            stdout = subprocess.check_output([cmd], shell=True)
            return stdout.decode()#.splitlines()

do_update = True
do_update = False

do_cleanup = False
do_cleanup = True

do_run = False
do_run = True

if cluster and do_cleanup:
    for cmd in [
        #"rm -fr results cache_dir ",
        #"find . -name *sparselets* -exec rm -fr {} \\;",
        "find . -name *lock* -exec rm -fr {} \\;",
        #"touch frioul; rm frioul* ",
        ]:
        print(k.run_on_cluster(cmd))

# preparing
experiments = ['experiment_' + name +  '.py' for name in ['ms', 'animals', 'testing']]
if do_run:
    # RUNNING
    k.run_cmd ("rsync -av ../database ../test")
    if cluster: k.push_to_cluster()
    for experiment in experiments:
        if cluster:
            fullcmd = 'ipython {experiment}'.format(experiment=experiment)
            for cmd in [
                "frioul_batch  -M {N_jobs} '{cmd}' ".format(N_jobs=k.N_jobs, cmd=fullcmd), 
                "frioul_list_jobs -v |grep job_array_id |uniq -c",
                        ]:
                print(k.run_on_cluster(cmd))
        else:
            fullcmd = 'ipython3 {experiment}'.format(experiment=experiment)
            run_cmd (fullcmd)
            

In [ ]:
# GETTING the data
import time, os
if cluster:
    while True:    
        print(k.pull_from_cluster())
        print(k.run_on_cluster("tail -n 10 {}".format(os.path.join(k.PATH, 'debug.log'))))
        print(k.run_on_cluster("frioul_list_jobs -v |grep job_array_id |uniq -c"))
        locks = k.run_cmd ("find . -name *lock -exec ls -l {} \;")
        print(locks)
        if len(locks) == 0: break
        time.sleep(100)               

In [10]:
! rsync -av -u --delete --exclude .AppleDouble --exclude .git perrinet.l@frioul.int.univ-amu.fr:/hpc/invibe/perrinet.l/science/PerrinetBednar15/{results,cache_dir,debug.log} ../test/ 

receiving incremental file list
deleting cache_dir/supp_SVM_serre07_distractors_serre07_targets_full_precomputed_KL_type_KL.npy_lock
cache_dir/

sent 46 bytes  received 110,173 bytes  44,087.60 bytes/sec
total size is 5,617,192,498  speedup is 50,963.92


## version control

In [ ]:
!git s